In [2]:
import random, string
import sys, os
import numpy as np

In [3]:
def read_input_file(file_path):
    
    try:
        with open(file_path, "r") as f:
            first_line = f.readline().strip().split(' ')
            num_vertices = int(first_line[0])
            num_edges = int(first_line[1])
            p = int(first_line[2])
            
            #edge_cost_map = {}
            cost_matrix = np.matrix(np.ones((num_vertices, num_vertices)) * np.inf)
            
            for line in f:
                line = line.strip().split(' ')
                #print(int(line[0]), int(line[1]), int(line[2]))
                #edge_cost_map[(int(line[0]), int(line[1]))] = int(line[2])
#                 if int(line[0]) in edge_cost_map:
#                     edge_cost_map[int(line[0])].append((int(line[1]), int(line[2])))
#                 else:
#                     edge_cost_map[int(line[0])] = [(int(line[1]), int(line[2]))]
                
                cost_matrix[int(line[0])-1, int(line[1])-1] = int(line[2])
        
            for i in range(0, num_vertices):
                cost_matrix[i, i] = 0
               
            # Floyd–Warshall algorithm for shortest path
            # https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm
            for k in range(0, num_vertices):
                for i in range(0, num_vertices):
                    for j in range(0, num_vertices):
                        if cost_matrix[i,j] > cost_matrix[i,k] + cost_matrix[k,j]: 
                            cost_matrix[i,j] = cost_matrix[i,k] + cost_matrix[k,j]
                        
        
            return num_vertices, num_edges, p, cost_matrix
        
    except IOError:
        return None
    

In [4]:
num_vertices, num_edges, p, cost_matrix = read_input_file('OR-Library/pmed3.txt')

In [5]:
cost_matrix

matrix([[  0.,  77., 139., ..., 251., 260., 299.],
        [370.,   0.,  62., ..., 264., 265., 304.],
        [493., 570.,   0., ..., 379., 388., 427.],
        ...,
        [114., 191., 253., ...,   0.,   9.,  48.],
        [105., 182., 244., ..., 350.,   0.,  39.],
        [ 66., 143., 205., ..., 317., 326.,   0.]])

In [15]:
#graph

{1: [(2, 77), (51, 49), (66, 62), (35, 71)],
 2: [(3, 62), (37, 75), (45, 61), (73, 67)],
 3: [(4, 90)],
 4: [(5, 22), (37, 32), (31, 53)],
 5: [(6, 17)],
 6: [(7, 19)],
 7: [(8, 11)],
 8: [(9, 4)],
 9: [(10, 3), (77, 10)],
 10: [(11, 16)],
 11: [(12, 67)],
 12: [(13, 35), (44, 1)],
 13: [(14, 21), (57, 53), (30, 38), (85, 79)],
 14: [(15, 14), (66, 49)],
 15: [(16, 43), (81, 44)],
 16: [(17, 11)],
 17: [(18, 91), (61, 72), (90, 76)],
 18: [(19, 55), (90, 84)],
 19: [(20, 33)],
 20: [(21, 80)],
 21: [(22, 90), (84, 33), (33, 30)],
 22: [(23, 81), (27, 9)],
 23: [(24, 91), (56, 26)],
 24: [(25, 23), (28, 37)],
 25: [(26, 49), (44, 42), (59, 38)],
 26: [(27, 62), (42, 51)],
 27: [(28, 96)],
 28: [(29, 25)],
 29: [(30, 92)],
 30: [(31, 77)],
 31: [(32, 41)],
 32: [(33, 40), (26, 15)],
 33: [(34, 97), (31, 57)],
 34: [(35, 1)],
 35: [(36, 14), (92, 54)],
 36: [(37, 18), (24, 6)],
 37: [(38, 69)],
 38: [(39, 72)],
 39: [(40, 17), (18, 87), (83, 95), (68, 34)],
 40: [(41, 56), (82, 76), (48,

In [5]:
#graph.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100])

In [6]:
class Chromosome:
    """
    Klasa Chromosome predstavlja jedan hromozom za koji se cuva njegov genetski kod i 
    vrednost funkcije prilagodjenosti.
    Genetski kod predstavlja potencijalno resenje problema, tj. listu cvorova koji su izabrani 
    kao lokacije za postavljanje objekta.
    """
    def __init__(self, content, fitness):
        self.content = content
        self.fitness = fitness
    def __str__(self): return "%s f=%d" % (self.content, self.fitness)
    def __repr__(self): return "%s f=%d" % (self.content, self.fitness)
    

In [45]:
class GeneticAlgorithm:
   
    def __init__(self,num_facilities, p, cost_matrix):
        
        self.num_facilities = num_facilities
        self.p = p
        self.cost_matrix = cost_matrix
    
        self.iterations = 100                             # Maksimalni dozvoljeni broj iteracija
        self.generation_size = 100                        # Broj jedinki u jednoj generaciji
        self.mutation_prob = 0.1                           # Verovatnoca da se desi mutacija
        self.reproduction_size = 40                      # Broj jedinki koji ucestvuje u reprodukciji
        self.current_iteration = 0                         # Koristi se za interno pracenje iteracija algoritma
        self.top_chromosome = None                         # Hromozom koji predstavlja resenje optimizacionog procesa
        self.hypermutation_prob = 0.5
        
    def mutation(self, chromosome):
        """Vrsi mutaciju nad hromozomom sa verovatnocom self.mutation_prob"""
        mp = random.random()
        #print(mp)
        if mp < self.mutation_prob:
            i = random.randint(0, len(chromosome)-1)
            # demand points bez trenutnih medijana:
            demand_points = [element for element in range(0,len(self.cost_matrix)) if element not in chromosome] 
            #print(demand_points)
            chromosome[i] = random.choice(demand_points)
            
        return chromosome
    
    
    def crossover(self, parent1, parent2):
        
        identical_elements = [element for element in parent1 if element in parent2]
        #print(identical_elements)
        if len(identical_elements) == len(parent1):
            return parent1, parent2  # TODO: parent2 treba da se izbrise iz populacije
        
        k = random.randint(1, len(parent1)-len(identical_elements))
        #print(k)
        
        child1 = []
        child2 = []

        #print(parent1)
        #print(parent2)
        
        exchange_vector_for_parent1 = [element for element in parent1 if element not in identical_elements]
        exchange_vector_for_parent2 = [element for element in parent2 if element not in identical_elements]   
        #print(exchange_vector_for_parent1)
        #print(exchange_vector_for_parent2)
        
        for i in range(k):
            tmp = exchange_vector_for_parent1[i]
            exchange_vector_for_parent1[i] = exchange_vector_for_parent2[i]
            exchange_vector_for_parent2[i] = tmp
        #print(exchange_vector_for_parent1)
        #print(exchange_vector_for_parent2)
        
        child1 = identical_elements + exchange_vector_for_parent1
        child2 = identical_elements + exchange_vector_for_parent2
        
        #print(parent1)
        #print(parent2)
        #print(child1)
        #print(child2)
        return child1, child2


    def cost_to_nearest_median(self, facility, medians):
        min_cost = self.cost_matrix[facility, medians[0]]
        #nearest_med = medians[0]
        for median in medians:
            if min_cost > self.cost_matrix[facility, median]:
                min_cost = self.cost_matrix[facility, median]
                #nearest_med = median        
        #return nearest_med
        return min_cost

    def fitness(self, chromosome):
        #print(self.cost_to_nearest_median(0, chromosome))
        cost_sum = 0
        for i in range(self.num_facilities):
            cost_sum += self.cost_to_nearest_median(i, chromosome)
            #cost_sum += self.cost_matrix[i, self.nearest_median(i, chromosome)]
        return cost_sum
    
    
    def initial_random_population(self):
        """Generise generation_size nasumicnih jedinki."""
        init_population = []
        for k in range(self.generation_size):
            rand_medians = []
            for i in range(self.p):
                rand_median = random.choice(range(self.num_facilities))
                rand_medians.append(rand_median)
            init_population.append(rand_medians)
        init_population = [Chromosome(content, self.fitness(content)) for content in init_population]
        self.top_chromosome = min(init_population, key=lambda chromo: chromo.fitness)
        print("Current top solution: %s" % self.top_chromosome)
        return init_population
    
    
    def selection(self, chromosomes):
        """Ranking-based selection method"""
        #print(chromosomes)
        # Hromozomi se sortiraju u rastucem poretku po vrednosti fitnes funkcije
        chromosomes.sort(key=lambda x: x.fitness)
        #print(chromosomes)
        L = self.reproduction_size
        selected_chromosomes = []
        
        for i in range(self.reproduction_size):
            j = L - np.floor((-1 + np.sqrt(1 + 4*random.uniform(0, 1)*(L**2 + L))) / 2)
            selected_chromosomes.append(chromosomes[int(j)])
        return selected_chromosomes
    
    
    def create_generation(self, for_reproduction):
        """
        Od jedinki dobijenih u okviru 'for_reproduction' generise novu generaciju
        primenjujuci genetske operatore 'crossover' i 'mutation'.
        Nova generacija je iste duzine kao i polazna.
        """
        new_generation = []
       
        while len(new_generation) < self.generation_size:
            parents = random.sample(for_reproduction, 2)
            child1, child2 = self.crossover(parents[0].content, parents[1].content)

            self.mutation(child1)
            self.mutation(child2)

            new_generation.append(Chromosome(child1, self.fitness(child1)))
            new_generation.append(Chromosome(child2, self.fitness(child2)))

        return new_generation
    
    
    
    def optimize(self):
        
        chromosomes = self.initial_random_population()

        while self.current_iteration < self.iterations:
            print("Iteration: %d" % self.current_iteration)

            # Izaberemo iz populacije skup jedinki za reprodukciju
            for_reproduction = self.selection(chromosomes)

            # Primenom operatora ukrstanja i mutacije kreiraj nove jedinke
            # i izracunaj njihovu prilagodjenost.
            # Dobijene jedinke predstavljaju novu generaciju.
            chromosomes = self.create_generation(for_reproduction)
            self.current_iteration += 1
            
            chromosome_with_min_fitness = min(chromosomes, key=lambda chromo: chromo.fitness)
            if chromosome_with_min_fitness.fitness < self.top_chromosome.fitness:
                self.top_chromosome = chromosome_with_min_fitness
            print("Current top solution: %s" % self.top_chromosome)
            print()
            
        print()
        print("Final top solution: %s" % self.top_chromosome)
    
#     def hypermutation(self, chromosomes):
        
#         # Uzima se 10% populacije
#         k = self.generation_size / 10
#         rand_subset = random.sample(chromosomes, k)
#         print(rand_subset)
        

    

In [46]:
genetic = GeneticAlgorithm(num_vertices, p, cost_matrix)
genetic.optimize()

Current top solution: [62, 71, 98, 43, 56, 82, 26, 52, 66, 49] f=9177
Iteration: 0
Current top solution: [41, 95, 80, 72, 56, 20, 76, 48, 36, 26] f=9135

Iteration: 1
Current top solution: [83, 56, 70, 76, 42, 30, 25, 23, 16, 64] f=8675

Iteration: 2
Current top solution: [20, 36, 76, 48, 95, 25, 41, 72, 56, 30] f=8572

Iteration: 3
Current top solution: [36, 25, 56, 76, 2, 5, 51, 85, 95, 26] f=8277

Iteration: 4
Current top solution: [25, 56, 39, 10, 20, 26, 70, 30, 35, 98] f=8196

Iteration: 5
Current top solution: [56, 20, 25, 76, 36, 98, 70, 12, 26, 95] f=8146

Iteration: 6
Current top solution: [56, 25, 36, 20, 76, 95, 26, 39, 10, 98] f=7776

Iteration: 7
Current top solution: [56, 25, 36, 20, 76, 95, 26, 39, 10, 98] f=7776

Iteration: 8
Current top solution: [56, 25, 36, 20, 76, 95, 26, 39, 10, 98] f=7776

Iteration: 9
Current top solution: [25, 56, 20, 76, 95, 98, 36, 55, 26, 69] f=7720

Iteration: 10
Current top solution: [25, 56, 98, 20, 36, 39, 76, 26, 95, 69] f=7643

Iterati

Current top solution: [62, 95, 25, 56, 28, 32, 63, 36, 65, 69] f=6970

Iteration: 97
Current top solution: [62, 95, 25, 56, 28, 32, 63, 36, 65, 69] f=6970

Iteration: 98
Current top solution: [62, 95, 25, 56, 28, 32, 63, 36, 65, 69] f=6970

Iteration: 99
Current top solution: [62, 95, 25, 56, 28, 32, 63, 36, 65, 69] f=6970


Final top solution: [62, 95, 25, 56, 28, 32, 63, 36, 65, 69] f=6970


In [18]:
genetic = GeneticAlgorithm(num_vertices, p, cost_matrix)
#genetic.mutation([1,23,47,80])
#genetic.crossover([24,12,9,26,18,40], [8,13,18,36,24,20])
#genetic.crossover([1,23,47,80,40,65,77], [2,50, 23,65,67,89,99])   
#genetic.crossover([1,2,3,4], [1,2,3,4])
genetic.fitness([65,50,90])
#init_pop = genetic.initial_random_population()
#init_pop

49.0


21574.0

In [13]:
for_reproduction = genetic.selection(init_pop)
for_reproduction

[[72, 12, 49, 70, 79, 5, 76, 87, 21, 25] f=15618,
 [56, 96, 47, 93, 79, 6, 19, 27, 88, 84] f=19341,
 [22, 67, 50, 62, 56, 90, 21, 63, 84, 74] f=19252,
 [51, 86, 55, 90, 95, 95, 93, 96, 29, 29] f=16154,
 [42, 41, 6, 69, 89, 73, 74, 17, 42, 56] f=16183,
 [72, 12, 49, 70, 79, 5, 76, 87, 21, 25] f=15618,
 [22, 67, 50, 62, 56, 90, 21, 63, 84, 74] f=19252,
 [51, 86, 55, 90, 95, 95, 93, 96, 29, 29] f=16154,
 [40, 59, 27, 33, 39, 74, 74, 98, 18, 40] f=19498,
 [67, 62, 83, 60, 83, 74, 70, 3, 23, 74] f=20112]

In [15]:
new_generation = genetic.create_generation(for_reproduction)
new_generation

[[67, 62, 74, 74, 22, 60, 83, 70, 3, 23] f=19524,
 [67, 62, 74, 74, 83, 50, 56, 90, 21, 63, 84] f=20986,
 [42, 41, 6, 69, 79, 5, 76, 87, 21, 25] f=16885,
 [72, 12, 49, 70, 89, 73, 74, 17, 42, 56] f=18106,
 [67, 62, 83, 60, 83, 6, 19, 27, 88, 84] f=21373,
 [56, 96, 47, 93, 79, 74, 70, 3, 23, 74] f=17990,
 [8, 51, 86, 50, 62, 56, 21, 63, 84, 74] f=20490,
 [90, 22, 67, 55, 95, 95, 93, 96, 29, 29] f=22563,
 [93, 96, 56, 47, 79, 6, 19, 95, 29, 29] f=19973,
 [93, 96, 51, 86, 55, 90, 95, 27, 88, 84] f=20862,
 [67, 62, 74, 83, 60, 83, 70, 21, 63, 84] f=22110,
 [67, 62, 74, 22, 50, 56, 90, 3, 23] f=19653,
 [51, 86, 32, 69, 89, 73, 74, 17, 42, 56] f=14687,
 [42, 41, 55, 90, 95, 95, 93, 96, 29, 29] f=17949,
 [74, 74, 42, 41, 6, 69, 89, 73, 17, 40] f=28580,
 [74, 74, 40, 59, 27, 33, 39, 98, 18, 42, 56] f=25058,
 [42, 12, 49, 70, 79, 5, 76, 87, 21, 25] f=17119,
 [72, 41, 6, 69, 89, 73, 74, 17, 42, 56] f=17720,
 [74, 74, 40, 59, 27, 33, 39, 98, 3, 23] f=22332,
 [74, 74, 67, 62, 83, 60, 83, 70, 18, 4

In [44]:
#genetic.hypermutation(new_generation)